# IQ Data Analysis in Octave
This notebook demonstrates how to process raw I/Q samples:
- Load binary data from a `.dat` file
- Compute and visualize the Power Spectral Density (PSD)
- Inspect time-domain samples
- Plot amplitude histograms
- Save cleaned data for later use

Before running any code, specify your sampling rate and file path in the cells below.

## 1. Set Sampling Frequency
Replace the blank assignment with your sampling rate in Hz.

In [ ]:
% Sampling frequency (Hz)
fs = ;  % ← Enter sampling frequency, e.g. 2.046e6

## 2. Set Data File Path
Provide the relative or absolute path to your I/Q `.dat` file.

In [ ]:
% Path to I/Q data file
data_file = '';  % ← Enter filename, e.g. 'test1.dat'

## 3. Load and Format I/Q Samples
Read the binary file, handle interleaved I/Q channels, and remove initial transients.

In [ ]:
% === Fixed Parameters ===
fi = 0;                   % Intermediate frequency (Hz)
IsComplex = true;         % True if data is I/Q
Is8Bits = false;          % 8-bit integer format
Is16Bits = false;         % 16-bit integer format
Is32BitsFloat = true;     % 32-bit float format
secondOfData = 1;         % Seconds of data to read
fl1 = 1575.42e6;          % Center frequency (Hz) for labels
% ========================

% Read binary samples
fid = fopen(data_file, 'r');
if IsComplex
    if Is16Bits
        [raw, ~] = fread(fid, 2*secondOfData*fs, 'int16');
    elseif Is8Bits
        [raw, ~] = fread(fid, 2*secondOfData*fs, 'int8');
    else
        [raw, ~] = fread(fid, 2*secondOfData*fs, 'float32');
    end
    data = raw(1:2:end) + 1i*raw(2:2:end);
else
    if Is16Bits
        [data, ~] = fread(fid, secondOfData*fs, 'int16');
    elseif Is8Bits
        [data, ~] = fread(fid, secondOfData*fs, 'int8');
    else
        [data, ~] = fread(fid, secondOfData*fs, 'float32');
    end
end
fclose(fid);

% Remove initial transient samples
if length(data) > 1e6
    data(1:1e6) = [];
end

## 4. Power Spectral Density
Use Welch's method to estimate PSD and plot it. Frequency axis is in MHz.

In [ ]:
% Compute PSD
[pw, f] = pwelch(data, [], [], [], fs);

% Plot PSD
figure;
if IsComplex
    plot((f + fl1 - fs/2)/1e6, 10*log10(fftshift(pw)));
else
    plot(f/1e6, 10*log10(fftshift(pw)));
end
grid on;
xlabel('Frequency (MHz)');
ylabel('PSD (dBW/Hz)');
title('Power Spectral Density');

## 5. Time-Domain View
Visualize the first 100,000 samples of the real and imaginary parts.

In [ ]:
% Real part
figure;
plot(real(data(1:1e5)), 'r.');
xlabel('Sample');
ylabel('Amplitude');
title('Real Part');

% Imaginary part
figure;
plot(imag(data(1:1e5)), 'b.');
xlabel('Sample');
ylabel('Amplitude');
title('Imaginary Part');

## 6. Histogram of Real Component
Plot the amplitude distribution to check for clipping or quantization.

In [ ]:
figure;
hist(real(data(1:1e5)), 18);
xlabel('Amplitude');
ylabel('Count');
title('Histogram of Real Part');

## 7. Save Processed Data
Store the cleaned complex samples for later analysis.

In [ ]:
% Save to MAT-file
save('IQ_samples.mat', 'data');
disp('Data saved to IQ_samples.mat');